In [35]:
import os
import subprocess
import pandas as pd
import platform
import pathlib
import numpy as np
from annotations import *
from extract_video_features import *
from extract_audio_features import *

In [36]:
#get current directory
cwd = os.getcwd()

In [37]:
cwd

'/Users/valentinadiproietto/filrouge'

In [38]:
OpenFace_folder = '/Users/valentinadiproietto/OpenFace'

In [39]:
Video_folder = '/Users/valentinadiproietto/Desktop/video_stress'

In [40]:
new_videos = ['/Users/valentinadiproietto/Desktop/video_stress/WIN_20210403_18_49_15_Pro.mp4']

In [12]:
for i in new_videos:
    extract_video_features(OpenFace_folder, i)

In [41]:
video_paths, video_names = get_videos(Video_folder)

In [42]:
video_paths

['/Users/valentinadiproietto/Desktop/video_stress/WIN_20210331_21_22_52_Pro.mp4',
 '/Users/valentinadiproietto/Desktop/video_stress/WIN_20210329_14_13_45_Pro.mp4',
 '/Users/valentinadiproietto/Desktop/video_stress/WIN_20210406_18_49_10_Pro.mp4',
 '/Users/valentinadiproietto/Desktop/video_stress/WIN_20210408_14_11_32_Pro.mp4',
 '/Users/valentinadiproietto/Desktop/video_stress/WIN_20210408_15_20_51_Pro.mp4',
 '/Users/valentinadiproietto/Desktop/video_stress/WIN_20210404_10_58_27_Pro.mp4',
 '/Users/valentinadiproietto/Desktop/video_stress/WIN_20210414_06_24_52_Pro.mp4',
 '/Users/valentinadiproietto/Desktop/video_stress/WIN_20210406_15_06_15_Pro.mp4',
 '/Users/valentinadiproietto/Desktop/video_stress/WIN_20210417_14_53_12_Pro.mp4',
 '/Users/valentinadiproietto/Desktop/video_stress/WIN_20210413_15_38_01_Pro.mp4',
 '/Users/valentinadiproietto/Desktop/video_stress/WIN_20210408_11_48_58_Pro.mp4',
 '/Users/valentinadiproietto/Desktop/video_stress/WIN_20210408_16_04_32_Pro.mp4',
 '/Users/valenti

In [43]:
video_names

['WIN_20210331_21_22_52_Pro',
 'WIN_20210329_14_13_45_Pro',
 'WIN_20210406_18_49_10_Pro',
 'WIN_20210408_14_11_32_Pro',
 'WIN_20210408_15_20_51_Pro',
 'WIN_20210404_10_58_27_Pro',
 'WIN_20210414_06_24_52_Pro',
 'WIN_20210406_15_06_15_Pro',
 'WIN_20210417_14_53_12_Pro',
 'WIN_20210413_15_38_01_Pro',
 'WIN_20210408_11_48_58_Pro',
 'WIN_20210408_16_04_32_Pro',
 'WIN_20210329_10_16_02_Pro',
 'WIN_20210323_19_17_40_Pro',
 'WIN_20210409_10_26_11_Pro',
 'Test_pour_AFPA',
 'WIN_20210405_15_09_16_Pro',
 'WIN_20210407_14_54_56_Pro_edit2',
 'WIN_20210406_21_05_52_Pro',
 'WIN_20210403_18_49_15_Pro',
 'WIN_20210408_14_02_19_Pro',
 'WIN_20210415_15_41_24_Pro',
 'WIN_20210406_18_35_52_Pro',
 'WIN_20210402_14_27_50_Pro',
 'WIN_20210407_09_04_05_Pro',
 'WIN_20210402_19_04_53_Pro',
 'WIN_20210416_08_06_54_Pro',
 'Video_1',
 'WIN_20210408_14_00_44_Pro',
 'WIN_20210404_21_41_12_Pro',
 'WIN_20210330_13_10_29_Pro']

In [44]:
list_dataframes =[]
for i in video_names:
    list_dataframes.append( create_dataframe_video('/Users/valentinadiproietto/OpenFace/processed/', i))
    

In [19]:
df=list_dataframes[0]

In [21]:
pd.set_option('display.max_columns', None)
df.head()

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,eye_lmk_x_0,eye_lmk_x_1,eye_lmk_x_2,eye_lmk_x_3,eye_lmk_x_4,eye_lmk_x_5,eye_lmk_x_6,eye_lmk_x_7,eye_lmk_x_8,eye_lmk_x_9,eye_lmk_x_10,eye_lmk_x_11,eye_lmk_x_12,eye_lmk_x_13,eye_lmk_x_14,eye_lmk_x_15,eye_lmk_x_16,eye_lmk_x_17,eye_lmk_x_18,eye_lmk_x_19,eye_lmk_x_20,eye_lmk_x_21,eye_lmk_x_22,eye_lmk_x_23,eye_lmk_x_24,eye_lmk_x_25,eye_lmk_x_26,eye_lmk_x_27,eye_lmk_x_28,eye_lmk_x_29,eye_lmk_x_30,eye_lmk_x_31,eye_lmk_x_32,eye_lmk_x_33,eye_lmk_x_34,eye_lmk_x_35,eye_lmk_x_36,eye_lmk_x_37,eye_lmk_x_38,eye_lmk_x_39,eye_lmk_x_40,eye_lmk_x_41,eye_lmk_x_42,eye_lmk_x_43,eye_lmk_x_44,eye_lmk_x_45,eye_lmk_x_46,eye_lmk_x_47,eye_lmk_x_48,eye_lmk_x_49,eye_lmk_x_50,eye_lmk_x_51,eye_lmk_x_52,eye_lmk_x_53,eye_lmk_x_54,eye_lmk_x_55,eye_lmk_y_0,eye_lmk_y_1,eye_lmk_y_2,eye_lmk_y_3,eye_lmk_y_4,eye_lmk_y_5,eye_lmk_y_6,eye_lmk_y_7,eye_lmk_y_8,eye_lmk_y_9,eye_lmk_y_10,eye_lmk_y_11,eye_lmk_y_12,eye_lmk_y_13,eye_lmk_y_14,eye_lmk_y_15,eye_lmk_y_16,eye_lmk_y_17,eye_lmk_y_18,eye_lmk_y_19,eye_lmk_y_20,eye_lmk_y_21,eye_lmk_y_22,eye_lmk_y_23,eye_lmk_y_24,eye_lmk_y_25,eye_lmk_y_26,eye_lmk_y_27,eye_lmk_y_28,eye_lmk_y_29,eye_lmk_y_30,eye_lmk_y_31,eye_lmk_y_32,eye_lmk_y_33,eye_lmk_y_34,eye_lmk_y_35,eye_lmk_y_36,eye_lmk_y_37,eye_lmk_y_38,eye_lmk_y_39,eye_lmk_y_40,eye_lmk_y_41,eye_lmk_y_42,eye_lmk_y_43,eye_lmk_y_44,eye_lmk_y_45,eye_lmk_y_46,eye_lmk_y_47,eye_lmk_y_48,eye_lmk_y_49,eye_lmk_y_50,eye_lmk_y_51,eye_lmk_y_52,eye_lmk_y_53,eye_lmk_y_54,eye_lmk_y_55,eye_lmk_X_0,eye_lmk_X_1,eye_lmk_X_2,eye_lmk_X_3,eye_lmk_X_4,eye_lmk_X_5,eye_lmk_X_6,eye_lmk_X_7,eye_lmk_X_8,eye_lmk_X_9,eye_lmk_X_10,eye_lmk_X_11,eye_lmk_X_12,eye_lmk_X_13,eye_lmk_X_14,eye_lmk_X_15,eye_lmk_X_16,eye_lmk_X_17,eye_lmk_X_18,eye_lmk_X_19,eye_lmk_X_20,eye_lmk_X_21,eye_lmk_X_22,eye_lmk_X_23,eye_lmk_X_24,eye_lmk_X_25,eye_lmk_X_26,eye_lmk_X_27,eye_lmk_X_28,eye_lmk_X_29,eye_lmk_X_30,eye_lmk_X_31,eye_lmk_X_32,eye_lmk_X_33,eye_lmk_X_34,eye_lmk_X_35,eye_lmk_X_36,eye_lmk_X_37,eye_lmk_X_38,eye_lmk_X_39,eye_lmk_X_40,eye_lmk_X_41,eye_lmk_X_42,eye_lmk_X_43,eye_lmk_X_44,eye_lmk_X_45,eye_lmk_X_46,eye_lmk_X_47,eye_lmk_X_48,eye_lmk_X_49,eye_lmk_X_50,eye_lmk_X_51,eye_lmk_X_52,eye_lmk_X_53,eye_lmk_X_54,eye_lmk_X_55,eye_lmk_Y_0,eye_lmk_Y_1,eye_lmk_Y_2,eye_lmk_Y_3,eye_lmk_Y_4,eye_lmk_Y_5,eye_lmk_Y_6,eye_lmk_Y_7,eye_lmk_Y_8,eye_lmk_Y_9,eye_lmk_Y_10,eye_lmk_Y_11,eye_lmk_Y_12,eye_lmk_Y_13,eye_lmk_Y_14,eye_lmk_Y_15,eye_lmk_Y_16,eye_lmk_Y_17,eye_lmk_Y_18,eye_lmk_Y_19,eye_lmk_Y_20,eye_lmk_Y_21,eye_lmk_Y_22,eye_lmk_Y_23,eye_lmk_Y_24,eye_lmk_Y_25,eye_lmk_Y_26,eye_lmk_Y_27,eye_lmk_Y_28,eye_lmk_Y_29,eye_lmk_Y_30,eye_lmk_Y_31,eye_lmk_Y_32,eye_lmk_Y_33,eye_lmk_Y_34,eye_lmk_Y_35,eye_lmk_Y_36,eye_lmk_Y_37,eye_lmk_Y_38,eye_lmk_Y_39,eye_lmk_Y_40,eye_lmk_Y_41,eye_lmk_Y_42,eye_lmk_Y_43,eye_lmk_Y_44,eye_lmk_Y_45,eye_lmk_Y_46,eye_lmk_Y_47,eye_lmk_Y_48,eye_lmk_Y_49,eye_lmk_Y_50,eye_lmk_Y_51,eye_lmk_Y_52,eye_lmk_Y_53,eye_lmk_Y_54,eye_lmk_Y_55,eye_lmk_Z_0,eye_lmk_Z_1,eye_lmk_Z_2,eye_lmk_Z_3,eye_lmk_Z_4,eye_lmk_Z_5,eye_lmk_Z_6,eye_lmk_Z_7,eye_lmk_Z_8,eye_lmk_Z_9,eye_lmk_Z_10,eye_lmk_Z_11,eye_lmk_Z_12,eye_lmk_Z_13,eye_lmk_Z_14,eye_lmk_Z_15,eye_lmk_Z_16,eye_lmk_Z_17,eye_lmk_Z_18,eye_lmk_Z_19,eye_lmk_Z_20,eye_lmk_Z_21,eye_lmk_Z_22,eye_lmk_Z_23,eye_lmk_Z_24,eye_lmk_Z_25,eye_lmk_Z_26,eye_lmk_Z_27,eye_lmk_Z_28,eye_lmk_Z_29,eye_lmk_Z_30,eye_lmk_Z_31,eye_lmk_Z_32,eye_lmk_Z_33,eye_lmk_Z_34,eye_lmk_Z_35,eye_lmk_Z_36,eye_lmk_Z_37,eye_lmk_Z_38,eye_lmk_Z_39,eye_lmk_Z_40,eye_lmk_Z_41,eye_lmk_Z_42,eye_lmk_Z_43,eye_lmk_Z_44,eye_lmk_Z_45,eye_lmk_Z_46,eye_lmk_Z_47,eye_lmk_Z_48,eye_lmk_Z_49,eye_lmk_Z_50,eye_lmk_Z_51,eye_lmk_Z_52,eye_lmk_Z_53,eye_lmk_Z_54,eye_lmk_Z_55,pose_Tx,pose_Ty,pose_Tz,pose_Rx,pose_Ry,pose_Rz,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19,x_20,x_21,x_22,x_23,x_24,x_25,x_26,x_27,x_28,x_29,x_30,x_31,x_32,x_33,x_34,x_35,x_36,x_37,x_38,x_39,x_40,x_41,x_42,x_43,x_44,x_45,x_46,x_47,x_48,x_4

In [22]:
len(list_dataframes)

31

In [23]:
for i in list_dataframes:
    print(check_success(i))
    

98.34157695664824
93.96474986647677
100.0
99.9474444882407
99.22779922779922
99.96171516079633
95.07823613086771
99.87678987678987
100.0
99.51358405504804
98.44551626678135
99.9315399484536
100.0
99.96298428888706
98.88920191572423
100.0
99.88188260948573
100.0
98.7196683331301
99.5310891693035
99.97950399672064
100.0
99.69153483453047
47.2531935317138
100.0
99.9057048561999
100.0
100.0
100.0
99.08576533650982
99.39536601126188


The video "'WIN_20210402_14_27_50_Pro'" is problematic for OpenFace, in less than 50% of the frames OpenFace succeded in detecting a face, but in all the other cases OpenFace was succesful, what do we do with this video, maybe we erase the video?

In [31]:
annotations_file ='/Users/valentinadiproietto/Desktop/video_stress/Videos_Annotations - Template.csv'

In [17]:
add_video_annotations(df, annotations_file, 2, 'WIN_20210331_21_22_52_Pro')

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,video_name,stress_global,stress,diapo
0,1,0,0.000,0.98,1,0.058144,-0.086946,-0.994515,-0.265938,-0.064276,...,1.0,0.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,0,1
1,2,0,0.060,0.98,1,0.076459,-0.143412,-0.986705,-0.259332,-0.081572,...,1.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,0,1
2,3,0,0.120,0.98,1,0.079903,-0.110330,-0.990678,-0.255499,-0.076939,...,1.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,0,1
3,4,0,0.180,0.98,1,0.117146,-0.106546,-0.987383,-0.247323,-0.062280,...,1.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,0,1
4,5,0,0.240,0.98,1,0.080384,-0.101181,-0.991615,-0.289122,-0.066332,...,1.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13743,13744,0,823.526,0.98,1,-0.273697,-0.041309,-0.960928,-0.439917,0.018487,...,0.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,1,18
13744,13745,0,823.586,0.98,1,-0.268158,-0.050530,-0.962049,-0.418280,0.013013,...,0.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,1,18
13745,13746,0,823.646,0.98,1,-0.237320,-0.039591,-0.970624,-0.385063,0.008704,...,0.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,1,18
13746,13747,0,823.705,0.98,1,-0.253612,-0.047883,-0.966120,-0.353438,0.000489,...,0.0,1.0,1.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,1,18


In [77]:
a,b, c =get_annotations_video(annotations_file, 'WIN_20210331_21_22_52_Pro','max')

In [79]:
print(a,b,c)

141 591 [0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]


In [18]:
 get_df_video_with_annotations('/Users/valentinadiproietto/OpenFace/processed/', 'WIN_20210331_21_22_52_Pro', annotations_file)

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,video_name,stress_global,stress,diapo
0,1,0,0.000,0.98,1,0.058144,-0.086946,-0.994515,-0.265938,-0.064276,...,1.0,0.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,0,1
1,2,0,0.060,0.98,1,0.076459,-0.143412,-0.986705,-0.259332,-0.081572,...,1.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,0,1
2,3,0,0.120,0.98,1,0.079903,-0.110330,-0.990678,-0.255499,-0.076939,...,1.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,0,1
3,4,0,0.180,0.98,1,0.117146,-0.106546,-0.987383,-0.247323,-0.062280,...,1.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,0,1
4,5,0,0.240,0.98,1,0.080384,-0.101181,-0.991615,-0.289122,-0.066332,...,1.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13743,13744,0,823.526,0.98,1,-0.273697,-0.041309,-0.960928,-0.439917,0.018487,...,0.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,1,18
13744,13745,0,823.586,0.98,1,-0.268158,-0.050530,-0.962049,-0.418280,0.013013,...,0.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,1,18
13745,13746,0,823.646,0.98,1,-0.237320,-0.039591,-0.970624,-0.385063,0.008704,...,0.0,1.0,0.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,1,18
13746,13747,0,823.705,0.98,1,-0.253612,-0.047883,-0.966120,-0.353438,0.000489,...,0.0,1.0,1.0,0.0,0.0,1.0,WIN_20210331_21_22_52_Pro,1,1,18


FROM THE THESIS

Nous avons choisi d’extraire la position et la rotation de la tête,l’intensité et la présence d’unités d’action faciales et la direction du regard résultant en un vecteur de taille 52. Comme différentes vidéos ont différentes fréquences d’images, nous avons décidé de lisser les valeurs avec une fenêtre temporelle de 0,5s et un chevauchement de 0,25s. 

In [24]:
df['AU01_r'].describe()

count    13748.000000
mean         0.326500
std          0.516159
min          0.000000
25%          0.000000
50%          0.020000
75%          0.510000
max          4.090000
Name: AU01_r, dtype: float64

In [27]:
df['AU01_c'].value_counts()

0.0    9978
1.0    3770
Name: AU01_c, dtype: int64

In [ ]:
For a description of the action units see here https://www.cs.cmu.edu/~face/facs.htm. To extract the action units OpenFace uses SVM. 


The columns 'AUXX_c' for XX in  ['01','02', '04', '05', '06', '07', '09', '10','12', '14', '15', '17', '20', '23', '25','26', '28', '45'] contain only 0's and 1's where 1 correspond to the presence of the corresponding AUXX, while 0 represents the absence. The columns 'AUXX_r' for XX in  for XX in  ['01','02', '04', '05', '06', '07', '09', '10','12', '14', '15', '17', '20', '23', '25','26', '45'] represent the intensity from 1 to 5 of the respective AU (with continuous value in between). REMARK: OpenFace does not compute the intesity for AU2 which is the lip suck. 

In [57]:
print(df.loc[df['AU01_c']==0].shape)

df.loc[df['AU01_c']==0].loc[df['AU01_r']!=0]['AU01_r']

(9978, 714)


1        0.01
2        0.04
3        0.04
4        0.03
8        0.01
         ... 
13709    0.04
13710    0.04
13721    0.05
13722    0.05
13723    0.05
Name: AU01_r, Length: 3422, dtype: float64

It is strange, from the above we can see that, even if 'AU01_c'=0, then there are almost one third of 'AU01_r' which are different from 0, why? So even if it is not detected OpenFace measures the intensity? The reason is that we have extracted the features AU in a dynamic way.

In [64]:
count =0
for i in df.columns:
    if i.startswith("AU"):
        count+=1
print(f"There are {count} columns corresponding to AU, {count-count//2} which mesure the intensity and {count//2} that measure if the AU is detected")               



There are 35 columns corresponding to AU, 18 which mesure the intensity and 17 that measure if the AU is detected


In [65]:
extract_video_features(OpenFace_folder, '/Users/valentinadiproietto/Desktop/video_stress/T2.mov')

CompletedProcess(args='build/bin/FeatureExtraction -f /Users/valentinadiproietto/Desktop/video_stress/T2.mov -2Dfp -3Dfp -pdmparams -pose -aus -gaze', returncode=0, stdout='Could not find the HAAR face detector location\nReading the landmark detector/tracker from: build/bin/model/main_ceclm_general.txt\nReading the landmark detector module from: build/bin/model/cen_general.txt\nReading the PDM module from: build/bin/model/pdms/In-the-wild_aligned_PDM_68.txt....Done\nReading the Triangulations module from: build/bin/model/tris_68.txt....Done\nReading the intensity CEN patch experts from: build/bin/model/patch_experts/cen_patches_0.25_of.dat....Done\nReading the intensity CEN patch experts from: build/bin/model/patch_experts/cen_patches_0.35_of.dat....Done\nReading the intensity CEN patch experts from: build/bin/model/patch_experts/cen_patches_0.50_of.dat....Done\nReading the intensity CEN patch experts from: build/bin/model/patch_experts/cen_patches_1.00_of.dat....Done\nReading part bas

In [66]:
T2= create_dataframe_video('/Users/valentinadiproietto/OpenFace/processed/', 'T2')

In [67]:
T2

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,0.324171,0.244781,-0.913781,0.256735,0.201875,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,0,0.075,0.98,1,0.347186,0.158946,-0.924228,0.261119,0.108721,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,0,0.150,0.98,1,0.303126,0.176676,-0.936430,0.267737,0.173383,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,4,0,0.225,0.98,1,0.220805,0.161280,-0.961891,0.124654,0.192952,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
4,5,0,0.300,0.98,1,0.194246,0.143095,-0.970460,0.068868,0.157785,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,0,5.247,0.98,1,0.426262,0.049787,-0.903229,0.064937,0.029549,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,72,0,5.322,0.98,1,0.410672,0.047468,-0.910547,0.218436,0.038879,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,73,0,5.397,0.98,1,0.388188,0.089885,-0.917186,0.247111,0.060966,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,74,0,5.472,0.98,1,0.338664,0.112290,-0.934183,0.251199,0.090329,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
T2.loc[T2['AU01_c']==0].loc[T2['AU01_r']!=0][['AU01_c', 'AU01_r']]

,AU01_c,AU01_r
1,0.0,0.07
2,0.0,0.07
3,0.0,0.36
4,0.0,0.29
5,0.0,0.29
16,0.0,0.07
17,0.0,0.18
18,0.0,0.21
19,0.0,0.19
20,0.0,0.08


In [71]:

def extract_video_features_static(OpenFace_directory, Video_path):
    '''
    Extract video features (csv) will be located in the 'processed' folder of OpenFace
    
    Parameters
    ----------
    OpenFace_directory : String
    Directory where the file FeatureExtraction can be found
    
    Video_path : String
    Directory where the Video(e.g. mp4) file can be found
    '''
    
    OS = platform.system() #Returns OS name
    os.chdir(f'{OpenFace_directory}') #Move to the OpenFace directory
    
    if OS == 'Windows':
        cmd = f"FeatureExtraction.exe -f {Video_path} -2Dfp -3Dfp -pdmparams -pose -aus -gaze"
        
    elif OS == 'Darwin':
        cmd = f"build/bin/FeatureExtraction -f {Video_path} -2Dfp -3Dfp -pdmparams -pose -aus -gaze -au_static"
        
    elif OS == 'Linux':
        cmd = f"FeatureExtraction -f {Video_path} -2Dfp -3Dfp -pdmparams -pose -aus -gaze"
        
    return subprocess.run(cmd, shell=True, text=True, capture_output=True)

In [73]:
extract_video_features_static(OpenFace_folder, '/Users/valentinadiproietto/Desktop/video_stress/T2.mov')

CompletedProcess(args='build/bin/FeatureExtraction -f /Users/valentinadiproietto/Desktop/video_stress/T2.mov -2Dfp -3Dfp -pdmparams -pose -aus -gaze -au_static', returncode=0, stdout='Could not find the HAAR face detector location\nReading the landmark detector/tracker from: build/bin/model/main_ceclm_general.txt\nReading the landmark detector module from: build/bin/model/cen_general.txt\nReading the PDM module from: build/bin/model/pdms/In-the-wild_aligned_PDM_68.txt....Done\nReading the Triangulations module from: build/bin/model/tris_68.txt....Done\nReading the intensity CEN patch experts from: build/bin/model/patch_experts/cen_patches_0.25_of.dat....Done\nReading the intensity CEN patch experts from: build/bin/model/patch_experts/cen_patches_0.35_of.dat....Done\nReading the intensity CEN patch experts from: build/bin/model/patch_experts/cen_patches_0.50_of.dat....Done\nReading the intensity CEN patch experts from: build/bin/model/patch_experts/cen_patches_1.00_of.dat....Done\nReadi

In [74]:
T2_static= create_dataframe_video('/Users/valentinadiproietto/OpenFace/processed/', 'T2')

In [75]:
T2_static.loc[T2_static['AU01_c']==0].loc[T2_static['AU01_r']!=0][['AU01_c', 'AU01_r']]

,AU01_c,AU01_r
0,0.0,0.23
8,0.0,0.61
9,0.0,0.38
10,0.0,0.25
11,0.0,0.32
12,0.0,0.27
13,0.0,0.21
14,0.0,0.08
15,0.0,0.12
16,0.0,0.24


They are a little less, but it does not change a lot. So what is the reason? The reason is explained in the github "NOTE that the intensity and presence predictors have been trained separately and on slightly different datasets, this means that the predictions of both might not always be consistent (e.g. the presence model could be predicting AU as not being present, but the intensity model could be predicting its value above 1)."

ABOUT gaze: to extract it uses a Constrained Local Neural Field (CLNF) landmark detector with which it determined the eyes landmarks, and then comput the gaze direction for each eye and the angle. 

gaze_0_x, gaze_0_y, gaze_0_z Eye gaze direction vector in world coordinates for eye 0 (normalized), eye 0 is the leftmost eye in the image (think of it as a ray going from the left eye in the image in the direction of the eye gaze)

gaze_1_x, gaze_1_y, gaze_1_z Eye gaze direction vector in world coordinates for eye 1 (normalized), eye 1 is the rightmost eye in the image (think of it as a ray going from the right eye in the image in the direction of the eye gaze)

gaze_angle_x, gaze_angle_y Eye gaze direction in radians in world coordinates averaged for both eyes and converted into more easy to use format than gaze vectors. If a person is looking left-right this will results in the change of gaze_angle_x (from positive to negative) and, if a person is looking up-down this will result in change of gaze_angle_y (from negative to positive), if a person is looking straight ahead both of the angles will be close to 0 (within measurement error).

eye_lmk_x_0, eye_lmk_x_1,... eye_lmk_x55, eye_lmk_y_1,... eye_lmk_y_55 location of 2D eye region landmarks in pixels. The landmark index can be found below

eye_lmk_X_0, eye_lmk_X_1,... eye_lmk_X55, eye_lmk_Y_0,... eye_lmk_Z_55 location of 3D eye region landmarks in millimeters. The landmark index can be found below


THEREFORE WE CAN TAKE gaze_0_x, gaze_0_y, gaze_0_z, gaze_1_x, gaze_1_y, gaze_1_z, gaze_angle_x, gaze_angle_y

OR EVEN SIMPLER gaze_angle_x, gaze_angle_y, angle averaged over the two eyes

HEAD POSE, this is calculated from the facial landmarks extracted with a  Convolutional Experts Constrained Local Model (CE-CLM) 

pose_Tx, pose_Ty, pose_Tz the location of the head with respect to camera in millimeters (positive Z is away from the camera)

pose_Rx, pose_Ry, pose_Rz Rotation is in radians around X,Y,Z axes with the convention R = Rx \* Ry \* Rz, left-handed positive sign. This can be seen as pitch (Rx), yaw (Ry), and roll (Rz). The rotation is in world coordinates with camera being the origin. x-axis=left-right, z-axis =in-out, y-axis = up-down, the Rx, Rz, Ry represent the angle of rotation with respect to a given axis.

So to see how much the head moves, I think we need both.

The feature that starts with p are parameters used in the  point distribution model (PDM) which is used to detect facial landmark and the position of the face. They are parameters used by the model, so I think we can avoid to use them. The author says "Local parameters (34 non-rigid) are parameters controlling non-rigid face shape, in other words things like facial expression and face shape (long, vs round, vs wide faces etc.). In my thesis they are q in equation 4.6 and are explained in detail in section 4.2.2 (https://www.cl.cam.ac.uk/~tb346/pub/thesis/phd_thesis.pdf)

If you are familiar with Principal Component Analysis (PCA), they are the dimensionality reduced components of the 68 facial landmarks. So individually each can represent a particular basis which roughly corresponds to an expression or face shape, but are not always easily mapped to one. If you want to visualize them and see their effect, have a look at matlab_version/pdm_generation/visualise_PDMs.m, this will visualize the first 5 ps and what effect on the face they have.

The global parameters, on the other hand, control the placement of the face in the image and are responsible for translation (x and y), scale/size, and the rotation of the face."

In [15]:
pd.options.display.max_seq_items = 2000
df.columns

Index(['frame', 'face_id', 'timestamp', 'confidence', 'success', 'gaze_0_x',
       'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z',
       'gaze_angle_x', 'gaze_angle_y', 'eye_lmk_x_0', 'eye_lmk_x_1',
       'eye_lmk_x_2', 'eye_lmk_x_3', 'eye_lmk_x_4', 'eye_lmk_x_5',
       'eye_lmk_x_6', 'eye_lmk_x_7', 'eye_lmk_x_8', 'eye_lmk_x_9',
       'eye_lmk_x_10', 'eye_lmk_x_11', 'eye_lmk_x_12', 'eye_lmk_x_13',
       'eye_lmk_x_14', 'eye_lmk_x_15', 'eye_lmk_x_16', 'eye_lmk_x_17',
       'eye_lmk_x_18', 'eye_lmk_x_19', 'eye_lmk_x_20', 'eye_lmk_x_21',
       'eye_lmk_x_22', 'eye_lmk_x_23', 'eye_lmk_x_24', 'eye_lmk_x_25',
       'eye_lmk_x_26', 'eye_lmk_x_27', 'eye_lmk_x_28', 'eye_lmk_x_29',
       'eye_lmk_x_30', 'eye_lmk_x_31', 'eye_lmk_x_32', 'eye_lmk_x_33',
       'eye_lmk_x_34', 'eye_lmk_x_35', 'eye_lmk_x_36', 'eye_lmk_x_37',
       'eye_lmk_x_38', 'eye_lmk_x_39', 'eye_lmk_x_40', 'eye_lmk_x_41',
       'eye_lmk_x_42', 'eye_lmk_x_43', 'eye_lmk_x_44', 'eye_lmk_x_45',
       'eye_lm

In [16]:
pd.set_option('display.max_columns', None)
df.head()

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.00,0.98,1,0.058144,-0.086946,-0.994515,-0.265938,-0.064276,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2,0,0.06,0.98,1,0.076459,-0.143412,-0.986705,-0.259332,-0.081572,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,3,0,0.12,0.98,1,0.079903,-0.110330,-0.990678,-0.255499,-0.076939,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,4,0,0.18,0.98,1,0.117146,-0.106546,-0.987383,-0.247323,-0.062280,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,5,0,0.24,0.98,1,0.080384,-0.101181,-0.991615,-0.289122,-0.066332,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


In [17]:
eliminate_features(df)

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,0.058144,-0.086946,-0.994515,-0.265938,-0.064276,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2,0,0.060,0.98,1,0.076459,-0.143412,-0.986705,-0.259332,-0.081572,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,3,0,0.120,0.98,1,0.079903,-0.110330,-0.990678,-0.255499,-0.076939,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,4,0,0.180,0.98,1,0.117146,-0.106546,-0.987383,-0.247323,-0.062280,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,5,0,0.240,0.98,1,0.080384,-0.101181,-0.991615,-0.289122,-0.066332,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13743,13744,0,823.526,0.98,1,-0.273697,-0.041309,-0.960928,-0.439917,0.018487,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
13744,13745,0,823.586,0.98,1,-0.268158,-0.050530,-0.962049,-0.418280,0.013013,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
13745,13746,0,823.646,0.98,1,-0.237320,-0.039591,-0.970624,-0.385063,0.008704,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
13746,13747,0,823.705,0.98,1,-0.253612,-0.047883,-0.966120,-0.353438,0.000489,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [18]:
len(df.columns)

54